In [25]:
import pandas as pd

import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
import numpy as np

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', None)


In [26]:

# Load the dataset
data = pd.read_csv('train.csv') 
data['Order Date'] = pd.to_datetime(data['Order Date'], format='%d/%m/%Y')

# filter for past year
latest_date = data['Order Date'].max()
one_year_ago = latest_date - pd.DateOffset(years=1)
filtered_data = data[data['Order Date'] >= one_year_ago]


In [31]:
# by Sub-Category
# process sales, cumulative sales sorted by sales, and corresponding cumulative contribution
sales_by_subcat = filtered_data.groupby(['Sub-Category', 'Category'])['Sales'].sum().reset_index()
total_sales = sales_by_subcat['Sales'].sum()
sales_by_subcat = sales_by_subcat.sort_values(by='Sales', ascending=False)
sales_by_subcat['Cumulative Sales'] = sales_by_subcat['Sales'].cumsum()
sales_by_subcat['Contribution'] = (sales_by_subcat['Sales'] / total_sales) * 100
sales_by_subcat['Cumulative Contribution'] = (sales_by_subcat['Cumulative Sales'] / total_sales) * 100

#classification of categories A, B, C
def classify_product(contribution):
    if contribution <= 80:
        return 'A'
    elif contribution <= 95:
        return 'B'
    else:
        return 'C'

sales_by_subcat['Classification'] = sales_by_subcat['Cumulative Contribution'].apply(classify_product)

#categorized sub-categories
sales_by_subcat

,Sub-Category,Category,Sales,Cumulative Sales,Contribution,Cumulative Contribution,Classification
13,Phones,Technology,104988.0340,104988.0340,14.509977,14.509977,A
5,Chairs,Furniture,93362.0590,198350.0930,12.903197,27.413174,A
3,Binders,Office Supplies,71932.7070,270282.8000,9.941532,37.354706,A
14,Storage,Office Supplies,68710.5340,338993.3340,9.496208,46.850914,A
6,Copiers,Technology,62899.3880,401892.7220,8.693073,55.543988,A
16,Tables,Furniture,60893.5425,462786.2645,8.415853,63.959841,A
0,Accessories,Technology,58240.9700,521027.2345,8.049252,72.009093,A
11,Machines,Technology,43544.6750,564571.9095,6.018136,78.027228,A
1,Appliances,Office Supplies,40212.7220,604784.6315,5.557640,83.584868,B
4,Bookcases,Furniture,30024.2797,634808.9112,4.149536,87.734404,B


In [28]:
# by product
# process sales, cumulative sales sorted by sales, and corresponding cumulative contribution
sales_by_product = filtered_data.groupby(['Product Name', 'Category'])['Sales'].sum().reset_index()
total_sales = sales_by_product['Sales'].sum()
sales_by_product = sales_by_product.sort_values(by='Sales', ascending=False)
sales_by_product['Cumulative Sales'] = sales_by_product['Sales'].cumsum()
sales_by_product['Contribution'] = (sales_by_product['Sales'] / total_sales) * 100
sales_by_product['Cumulative Contribution'] = (sales_by_product['Cumulative Sales'] / total_sales) * 100

#classification of categories A, B, C
def classify_product(contribution):
    if contribution <= 80:
        return 'A'
    elif contribution <= 95:
        return 'B'
    else:
        return 'C'

sales_by_product['Classification'] = sales_by_product['Cumulative Contribution'].apply(classify_product)

#categorized sub-categories
sales_by_product

,Product Name,Category,Sales,Cumulative Sales,Contribution,Cumulative Contribution,Classification
337,Canon imageCLASS 2200 Advanced Copier,Technology,35699.8980,35699.8980,4.933940,4.933940,A
861,Martin Yale Chadless Opener Electric Letter Op...,Office Supplies,11825.9020,47525.8000,1.634411,6.568351,A
569,GBC DocuBind TL300 Electric Binding System,Office Supplies,10943.2780,58469.0780,1.512427,8.080778,A
670,Hewlett Packard LaserJet 3310 Copier,Technology,9239.8460,67708.9240,1.277002,9.357780,A
1138,Samsung Galaxy Mega 6.3,Technology,9239.7800,76948.7040,1.276993,10.634773,A
397,Cubify CubeX 3D Printer Triple Head Print,Technology,7999.9800,84948.6840,1.105645,11.740418,A
538,Fellowes PB500 Electric Punch Plastic Comb Bin...,Office Supplies,7371.7420,92320.4260,1.018819,12.759237,A
654,HON 5400 Series Task Chairs for Big and Tall,Furniture,7220.0940,99540.5200,0.997860,13.757098,A
566,GBC DocuBind P400 Electric Binding System,Office Supplies,7077.1480,106617.6680,0.978104,14.735202,A
813,Lexmark MX611dhe Monochrome Laser Printer,Technology,5609.9670,112227.6350,0.775331,15.510533,A
